In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path

In [3]:
# find root 
root = Path.cwd().parent

# define outpath
out_path = root / "reports" / "tables" 

# set start date on 2023-03-31 to enable log-growth computation from 03-04-2023 onwards
start, end = "2023-03-31", "2025-07-16"

# define tickers 
tickers = [
    "AAPL","AIQ","AMD","AMZN","ARKQ","AVGO","BOTZ",
    "GOOGL","IRBO","META","MSFT","NVDA","ROBO","TSLA","TSM"
]

# Prices (Adj Close)
px = yf.download(tickers + ["^GSPC"], start=start, end=end, auto_adjust=False)["Adj Close"]

# Risk-free: 3M T-Bill (^IRX), annualized percent yield
rf = yf.download("^IRX", start=start, end=end, auto_adjust=False)["Adj Close"]


# Asset & market daily log returns from prices
logrets = np.log(px).diff()

# Convert RF
rf_simple_daily = (rf / 100.0) / 252.0
rf_simple_daily = rf_simple_daily.reindex(logrets.index).ffill()
rf_log_daily = np.log1p(rf_simple_daily)

# Compute Excess log returns
excess_mkt_log = logrets["^GSPC"] - rf_log_daily.squeeze()
excess_assets_log = logrets[tickers].sub(rf_log_daily.squeeze(), axis=0)

# Combine and clean
df = pd.concat([excess_assets_log, excess_mkt_log.rename("MKT")], axis=1)
df = df.replace([np.inf, -np.inf], np.nan).dropna(how="any")

# run regressions
results = []
for t in tickers:
    # daily log excess return (asset)
    y = df[t]                  
    # daily log excess return
    X = sm.add_constant(df["MKT"])

    n = y.shape[0]
    # Newey–West lag rule 
    q = int(4 * (n / 100)**(2/9)) if n > 0 else 0

    model = sm.OLS(y, X, missing="drop").fit(
        cov_type="HAC",
        cov_kwds={"maxlags": q, "use_correction": True}
    )

    # daily log excess alpha
    alpha = model.params["const"]       # 
    beta = model.params["MKT"]
    t_alpha = model.tvalues["const"]
    t_beta = model.tvalues["MKT"]
    r2 = model.rsquared
    nobs = int(model.nobs)              

    # Alpha over the whole period: compounded simple %
    alpha_total_comp_pct = np.expm1(alpha * nobs) * 100.0

    results.append((t, alpha_total_comp_pct, beta, t_alpha, t_beta, r2, nobs))


# Table Cols
cols = ["Ticker","Alpha_total_comp_%","Beta","t(Alpha)","t(Beta)","R2","N"]
capm_df = pd.DataFrame(results, columns=cols).set_index("Ticker")

# Formatting
capm_df["Alpha_total_comp_%"] = capm_df["Alpha_total_comp_%"].round(2)
capm_df["Beta"] = capm_df["Beta"].round(3)
capm_df["t(Alpha)"] = capm_df["t(Alpha)"].round(2)
capm_df["t(Beta)"] = capm_df["t(Beta)"].round(2)
capm_df["R2"] = (capm_df["R2"] * 100).round(1)

print(capm_df.sort_values("Beta", ascending=False))

# save CSV
capm_df.to_csv(out_path / "capm_results_compounded_alpha.csv", encoding="utf-8", index=True)


# LaTeX export
latex = capm_df.rename(columns={
    "Alpha_total_comp_%": "$\\alpha$ (total, \\%, compounded)",
    "Beta": "$\\beta$",
    "t(Alpha)": "$t(\\alpha)$",
    "t(Beta)": "$t(\\beta)$",
    "R2": "$R^2$ (\\%)",
    "N": "$N$"
}).to_latex(
    escape=False,
    column_format="lrrrrrr",  # index + 6 numeric cols
    bold_rows=False
)

with open(out_path / "capm_results.tex","w",encoding="utf-8") as f:
    f.write(latex)



[*********************100%***********************]  16 of 16 completed
[*********************100%***********************]  1 of 1 completed


        Alpha_total_comp_%   Beta  t(Alpha)  t(Beta)    R2    N
Ticker                                                         
TSLA                -34.45  2.325     -0.54    16.02  34.5  572
NVDA                181.28  2.182      1.79    12.55  42.7  572
AMD                 -24.44  2.053     -0.51    17.72  41.4  572
AVGO                118.97  2.003      1.34    14.94  40.9  572
TSM                  45.22  1.582      0.95    15.38  39.9  572
ARKQ                  2.92  1.572      0.12    23.80  69.5  572
META                 89.69  1.508      1.58    18.67  43.4  572
IRBO                -23.25  1.444     -1.37    30.32  74.2  572
AMZN                 27.14  1.409      0.78    19.65  49.5  572
BOTZ                -24.25  1.333     -1.55    30.46  74.9  572
AIQ                   6.55  1.323      0.50    42.64  85.3  572
AAPL                -20.41  1.191     -0.76    11.38  49.3  572
ROBO                -31.36  1.190     -2.35    38.52  75.8  572
GOOGL                11.83  1.124      0

,AAPL,AIQ,AMD,AMZN,ARKQ,AVGO,BOTZ,GOOGL,IRBO,META,MSFT,NVDA,ROBO,TSLA,TSM,MKT
Date,,,,,,,,,,,,,,,,
2023-04-03,0.007491,-0.004257,-0.015086,-0.008738,-0.014400,0.001267,-0.001751,0.005874,-0.003053,0.005136,-0.003900,0.006564,-0.002377,-0.063301,-0.002118,0.003511
2023-04-04,-0.003443,-0.004484,-0.007359,0.014738,-0.009712,-0.012812,-0.015624,0.003256,-0.009176,0.007526,-0.000362,-0.018666,-0.010127,-0.011495,-0.007431,-0.006001
2023-04-05,-0.011541,-0.019022,-0.035323,-0.027987,-0.029318,-0.008735,-0.019519,-0.002577,-0.024339,-0.015391,-0.010125,-0.021242,-0.021122,-0.037536,-0.021792,-0.002682
2023-04-06,0.005294,0.004399,-0.001160,0.009264,-0.002706,-0.010366,-0.003854,0.036926,-0.001840,0.021424,0.025025,0.005600,-0.004920,-0.002670,0.000256,0.003386
2023-04-10,-0.016293,0.003752,0.031841,0.000885,0.009846,0.007408,0.007533,-0.018623,0.008703,-0.006459,-0.007800,0.019656,0.006429,-0.003168,-0.013804,0.000804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-09,0.005198,0.001890,0.004103,0.014224,0.008273,0.022027,0.012422,0.012710,0.010932,0.016496,0.013610,0.017671,0.007449,-0.006670,0.017148,0.005875
2025-07-10,0.005829,-0.002914,0.040535,-0.001427,0.017276,-0.009205,-0.007212,0.005478,-0.007152,-0.007757,-0.004208,0.007294,0.005589,0.046031,-0.009180,0.002574
2025-07-11,-0.006070,-0.008681,0.015387,0.012173,0.011548,-0.003879,-0.008500,0.014197,-0.004770,-0.013638,0.003494,0.004816,-0.007246,0.011510,0.002614,-0.003471


In [9]:
def avg_logret_period(df, start, end):
    """Compute mean daily log return (%) for a given subperiod."""
    sub = df.loc[start:end]
    return sub.mean() * 100  # in %

# define periods within overall bounds
periods = {
    "2023": ("2023-03-31", "2023-12-31"),
    "2024": ("2024-01-01", "2024-12-31"),
    "2025": ("2025-01-01", "2025-06-16"),
    "2023-2024": ("2023-03-31", "2024-12-31"),
    "2024-2025": ("2024-01-01", "2025-06-16"),
    "2023-2025": ("2023-03-31", "2025-06-16"),
}

# compute averages
avg_returns = {label: avg_logret_period(logrets, s, e) for label, (s, e) in periods.items()}

# combine to DataFrame (rows = period, columns = tickers + MKT)
avg_daily_pct = pd.DataFrame(avg_returns).T.round(4)

# save to CSV
avg_daily_pct.to_csv(out_path / "avg_daily_log_returns_by_period_pct.csv", encoding="utf-8")

# show summary
print("Average daily log returns (%):")
print(avg_daily_pct)

Average daily log returns (%):
Ticker       AAPL     AIQ     AMD    AMZN    ARKQ    AVGO    BOTZ   GOOGL  \
2023       0.0846  0.1272  0.2171  0.2053  0.0731  0.3028  0.0602  0.1583   
2024       0.1063  0.0857 -0.0790  0.1458  0.1158  0.2953  0.0459  0.1220   
2025      -0.2038  0.0805  0.0401 -0.0134  0.0739  0.0768 -0.0190 -0.0585   
2023-2024  0.0970  0.1034  0.0475  0.1712  0.0976  0.2985  0.0520  0.1375   
2024-2025  0.0103  0.0841 -0.0421  0.0965  0.1028  0.2277  0.0258  0.0661   
2023-2025  0.0355  0.0988  0.0460  0.1335  0.0927  0.2532  0.0375  0.0975   

Ticker       IRBO    META    MSFT    NVDA    ROBO    TSLA     TSM     MKT  
2023       0.0527  0.2728  0.1447  0.3077  0.0248  0.0960  0.0668  0.0793  
2024       0.0290  0.2012  0.0482  0.3960 -0.0051  0.1927  0.2601  0.0831  
2025       0.0438  0.1622  0.1168  0.0662  0.0176 -0.1810  0.0846  0.0225  
2023-2024  0.0391  0.2318  0.0895  0.3583  0.0076  0.1514  0.1775  0.0815  
2024-2025  0.0336  0.1891  0.0695  0.2939  0.0019